## 1.6.1 Stereo Calibration 
Stereo calibration and triangulation for 4 meters

### import  libraries

In [2]:
vscode = 1

In [3]:
if(vscode == 1):
    # for vscode
    %matplotlib qt
else:
    # for jupyter notebook
    from mpl_toolkits.mplot3d import axes3d
    import matplotlib.pyplot as plt

    %matplotlib notebook


In [4]:
import matplotlib.pyplot as plt
import numpy as np
import cv2 
from typing import Sequence
from calib_lib import *
DECIMALS = 2  # how many decimal places to use in print


In [5]:
F = 16                                       # focal length( in mm )
image_size = np.array([11,7])               # sensor size(in mm)
PX= image_size[0]/2.0                       # principal point x-coordinate
PY= image_size[1]/2.0                       # principal point y-coordinate
IMAGE_HEIGTH = image_size[1]
IMAGE_WIDTH = image_size[0]
THETA_X = 0                                 # roll angle
THETA_Y = 0                                 # pitch angle
THETA_Z = 0                         # yaw angle


# camera Right
THETA_X_R = 0                                 # roll angle
THETA_Y_R = 0                                 # pitch angle
THETA_Z_R= 0                                 # yaw angle
# camera Left
THETA_X_L = 0                                 # roll angle
THETA_Y_L = 0                                 # pitch angle
THETA_Z_L= 0                                 # yaw angle

C_L = np.array([0,0,0])                     # camera centre
C_R = np.array([500,0,0])

chess_dimx,chess_dimy = (13,9)
chess_sq_size = 44


In [6]:
def repeat_func(xrange,yrange,zrange):

    translst= []

    for z in zrange:
        for x in xrange:
            for y in yrange:
                translst.append([x,y,z])
                
    len(translst)
    #print(translst)

    # chessboard print
    chess_pts = []
    world_pts = create_chessboard(chess_dimx,chess_dimy,chess_sq_size)
    rotangles = [[0,0,0],[np.pi/4,0,np.pi/2],[0,np.pi/4,np.pi/2],[0,np.pi/4,-np.pi/2],[np.pi/4,0,-np.pi/2]]

    for i in range(len(translst)):
        for j in range(len(rotangles)):    
            chess_pts.append(get_chessboard_rot_trans(world_pts,rotangles[j][0],rotangles[j][1],rotangles[j][2],translst[i][0],translst[i][1],translst[i][2]))

    chess_pts_arr = np.array(chess_pts)
    print(chess_pts_arr.shape)

    pattern_size_x,pattern_size_y = (chess_dimx,chess_dimy)

    X_pattern = np.linspace(0, pattern_size_x,pattern_size_x + 1)*chess_sq_size
    Y_pattern = np.linspace(0, pattern_size_y,pattern_size_y + 1)*chess_sq_size
    zdata = np.zeros((pattern_size_x + 1,pattern_size_y + 1))
    xdata, ydata = np.meshgrid(X_pattern, Y_pattern)

    xdata_ = xdata.flatten()
    ydata_ = ydata.flatten()
    zdata_ = zdata.flatten()
    
    # homogeneous coordinates
    world_pts =([xdata_,ydata_,zdata_])
    world_pts_ = np.array(world_pts).T

    world_pts_arr = np.zeros((chess_pts_arr.shape[0],(pattern_size_x+1)*(pattern_size_y+1),3),np.float32)
    
    for i in range(chess_pts_arr.shape[0]):
        world_pts_arr[i,:,:] = world_pts_

    world_pts_arr = world_pts_arr.reshape(world_pts_arr.shape[0],1,(pattern_size_x+1)*(pattern_size_y+1),3)
    
    x_lst_R = []
    x_lst_L = []
    for i in range(chess_pts_arr.shape[0]):
        x_arr_R, X_arr_R, E_R, K_R, P_R = get_image_points(chess_pts_arr[i,:,:],PX,PY,thetax= 0,thetay = 0,thetaz = 0,trans_x= -C_R[0],trans_y= -C_R[1],trans_z= -C_R[2],F = F)
        x_arr_L, X_arr_L, E_L, K_L, P_L = get_image_points(chess_pts_arr[i,:,:],PX,PY,thetax= 0,thetay = 0,thetaz = 0,trans_x= -C_L[0],trans_y= -C_L[1],trans_z= -C_L[2],F = F)
        
        x_lst_R.append(x_arr_R)
        x_lst_L.append(x_arr_L)

    x_zhang_R = np.array(x_lst_R,np.float32)
    x_zhang_L = np.array(x_lst_L,np.float32)

    print("World points: ",world_pts_arr.shape)
    print("Image points: ",x_zhang_L.shape)


    return world_pts_arr,x_zhang_R,x_zhang_L,P_R,P_L


In [7]:
def calib(world_pts_arr,x_zhang_R,x_zhang_L):    
    ret_R, mtx_R, dist_R, rvecs_R, tvecs_R = cv2.calibrateCamera(world_pts_arr, x_zhang_R, (image_size[0],image_size[1]), None, None)
    ret_L, mtx_L, dist_L, rvecs_L, tvecs_L = cv2.calibrateCamera(world_pts_arr, x_zhang_L, (image_size[0],image_size[1]), None, None)

    stereo_flags = cv2.CALIB_FIX_INTRINSIC

    # Here we fix the intrinsic camara matrixes so that only Rot, Trns, Emat and Fmat are calculated.
    # Hence intrinsic parameters are the same 

    criteria_stereo= (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)


    # This step is performed to transformation between the two cameras and calculate Essential and Fundamenatl matrix
    retS, new_mtxL, distL, new_mtxR, distR, Rot, Trns, Emat, Fmat = cv2.stereoCalibrate(world_pts_arr, x_zhang_L, x_zhang_R, mtx_L, dist_L, mtx_R, dist_R, (image_size[0],image_size[1]), criteria_stereo, stereo_flags)

    return ret_R,mtx_R,ret_L,mtx_L,retS,new_mtxL,new_mtxR,Rot,Trns,Emat,Fmat

In [9]:
ret_R_list = []
mtx_R_list = []
ret_L_list = []
mtx_L_list = []
retS_list = []
new_mtxL_list = []
new_mtxR_list = []
Rot_list = []
Trns_list = []
Emat_list = []
Fmat_list = []
P_R_org_list = []
P_L_org_list = []

xrange_0 = np.linspace(-650,1100,1)
yrange_0 = np.linspace(-600,600,1)
zrange_0 = ([4000])

xrange_1 = np.linspace(-650,1100,2)
yrange_1 = np.linspace(-600,600,2)
zrange_1 = ([4000])

xrange_2 = np.linspace(-650,1100,4)
yrange_2 = np.linspace(-600,600,4)
zrange_2 = ([4000])

xrange_3 = np.linspace(-650,1100,6)
yrange_3 = np.linspace(-600,600,6)
zrange_3 = ([4000])

xrange_4 = np.linspace(-650,1100,8)
yrange_4 = np.linspace(-600,600,8)
zrange_4 = ([4000])

xrange_5 = np.linspace(-650,1100,10)
yrange_5 = np.linspace(-600,600,10)
zrange_5 = ([4000])


x_range = ([xrange_0,xrange_1,xrange_2,xrange_3,xrange_4])
y_range = ([yrange_0,yrange_1,yrange_2,yrange_3,yrange_4])
z_range = ([zrange_0,zrange_1,zrange_2,zrange_3,zrange_4])
#x_range = ([xrange_0])
#y_range = ([yrange_0])
#z_range = ([zrange_0])



for i in range(len(z_range)):

    world_pts_arr = 0
    x_zhang_R = 0
    x_zhang_L = 0
    
    world_pts_arr,x_zhang_R,x_zhang_L, P_R, P_L = repeat_func(x_range[i],y_range[i],z_range[i])

    ret_R,mtx_R,ret_L,mtx_L,retS,new_mtxL,new_mtxR,Rot,Trns,Emat,Fmat = calib(world_pts_arr,x_zhang_R,x_zhang_L)

    ret_R_list.append([ret_R])
    mtx_R_list.append([mtx_R])
    ret_L_list.append([ret_L])
    mtx_L_list.append([mtx_L])
    retS_list.append([retS])
    new_mtxL_list.append([new_mtxL])
    new_mtxR_list.append([new_mtxR])
    Rot_list.append([Rot])
    Trns_list.append([Trns])
    Emat_list.append([Emat])
    Fmat_list.append([Fmat])
    P_R_org_list.append(P_R)
    P_L_org_list.append(P_L)

(5, 140, 3)
World points:  (5, 1, 140, 3)
Image points:  (5, 140, 2)
(20, 140, 3)
World points:  (20, 1, 140, 3)
Image points:  (20, 140, 2)
(80, 140, 3)
World points:  (80, 1, 140, 3)
Image points:  (80, 140, 2)
(180, 140, 3)
World points:  (180, 1, 140, 3)
Image points:  (180, 140, 2)
(320, 140, 3)
World points:  (320, 1, 140, 3)
Image points:  (320, 140, 2)


In [10]:
P_L_list = []
P_R_list = []

for i in range(len(ret_R_list)):
    R1,R2,P_L_est,P_R_est,Q,roi_left,roi_right = cv2.stereoRectify(new_mtxL_list[i][0], ret_L_list[i][0], new_mtxR_list[i][0], ret_R_list[i][0], (image_size[0],image_size[1]), Rot_list[i][0], Trns_list[i][0],flags = cv2.CALIB_ZERO_DISPARITY)
    P_L_list.append(P_L_est)
    P_R_list.append(P_R_est)

In [14]:
XestOpenCV = np.zeros((6,4,1))
Xoriginal = np.zeros((6,4,1))

X_org_arr = np.zeros((6,4,1))
X_est_arr = np.zeros((6,4,1))

estError = np.zeros((6,1))

x_R = np.array([7.2, 0])
x_L = np.array([7.6, 0])


for i in range(len(ret_R_list)):

    XestOpenCV[i,:,:] = cv2.triangulatePoints(P_L_list[i],P_R_list[i], x_L, x_R)
    Xoriginal[i,:,:] =  cv2.triangulatePoints(P_L_org_list[i],P_R_org_list[i], x_L, x_R)

    X_est_arr[i,:,:] = XestOpenCV[i,:,:]/XestOpenCV[i,-1,:]
    Xoriginal[i,:,:] = Xoriginal[i,:,:]/Xoriginal[i,-1,:]

    estError[i,:] = np.sum(np.abs(X_est_arr[i,:,:]-Xoriginal[i,:,:]))


In [15]:
X_est_arr

array([[[ 2.62487267e+03],
        [-4.37499371e+03],
        [ 1.99999751e+04],
        [ 1.00000000e+00]],

       [[ 2.62481948e+03],
        [-4.37501126e+03],
        [ 2.00000262e+04],
        [ 1.00000000e+00]],

       [[ 2.62494805e+03],
        [-4.37500643e+03],
        [ 2.00000162e+04],
        [ 1.00000000e+00]],

       [[ 2.62495080e+03],
        [-4.37500423e+03],
        [ 2.00000064e+04],
        [ 1.00000000e+00]],

       [[ 2.62501449e+03],
        [-4.37500321e+03],
        [ 2.00000059e+04],
        [ 1.00000000e+00]],

       [[ 0.00000000e+00],
        [ 0.00000000e+00],
        [ 0.00000000e+00],
        [ 0.00000000e+00]]])

In [24]:
estError

array([[0.15852678],
       [0.21795413],
       [0.07453543],
       [0.05987829],
       [0.02363784],
       [0.        ]])

In [25]:
ret_R_list

[[3.79360778713432e-08],
 [1.80175586527126e-07],
 [1.6152687955533396e-07],
 [1.548089584069534e-07],
 [1.527661508761033e-07]]

In [30]:
nr_chess = [20,80,180,320]

fig = plt.figure(figsize = (20,8))
ax = fig.add_subplot(121)
ax.plot(nr_chess,ret_R_list[1:],'k-*',label = "2 meters")

ax.set_yscale('log')
ax.set_ylabel("Reprojection error (in mm)")
ax.set_xlabel("number of chessboards")
ax.set_title("Stereo camera calibration error")
ax.grid('--')
ax.legend()

ax1 = fig.add_subplot(122)
ax1.set_ylabel("Triangulation error (in mm)")
ax1.set_xlabel("number of chessboards")
ax1.set_title("Stereo camera calibration error")
ax1.plot(nr_chess,estError[1:5],'k-*',label = "2 meters")
ax1.grid('--')
ax1.legend()


### Save the variables

In [19]:
%store P_L_org_list,P_R_org_list,retS_list,new_mtxL_list,ret_L_list,new_mtxR_list,ret_R_list,Rot_list,Trns_list >"../calibration/distances/fourmeters.txt"


Writing 'P_L_org_list,P_R_org_list,retS_list,new_mtxL_list,ret_L_list,new_mtxR_list,ret_R_list,Rot_list,Trns_list' (tuple) to file '../calibration/fourmeters.txt'.
